<a href="https://colab.research.google.com/github/jamg-upv/genAI4ResearchTeaching/blob/main/asepuc25/ASEPUC_UBdocSchoolMay25_cribado_LLM_embeddingsv2_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este codigo funciona con un entorno de de computacion de 12GB RAM sin GPU.
Lo he probado con 4 categorias y 1300 abstracts a clasificar y con modelos de embeedings de 384 tokens no gasta más de 3Gb de ram. Con los 14 modelos (donde hay algunos de 1024 tokens) llega a gastar 6Gb (la ram depende de los tokens del modelo y no de la cantidad de objetos a embedd)
los modelos de 384 tokens consumen unos 1-3 minutos por cada 600 abstracts
Los modelosd e 720 tokens consumen unos 5-15 minutos por cada 600 abstracts


#Preparacion previa (ejecutar solo una vez)



## (solo ejecutar si tus datos están en github y quieres importarlos) conexion a github

In [ ]:
#conectar con repositorio github publico para acceso a los datasets (ajusta el nombre del repositorio al que corresponda)
!git clone https://github.com/jamg-upv/LLMforSLRscreening.git

# elije la carpeta del repositorio donde esta el archivo de datos   ACEDE-ECNwsDic24/
# input_path = '/content/LLMforSLRscreening/datasets/'
input_path = '/content/LLMforSLRscreening/ACEDE-ECNwsDic24/'

# Define el nombre del archivo #Receuerda: estructura del archivo de datos:
# Component	Description	Type	Class
input_file_name = 'ACEDE-ECN-workshop-dic-2024-HIWP3.csv'



output_path = '/content/'

Cloning into 'LLMforSLRscreening'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 460 (delta 93), reused 20 (delta 20), pack-reused 317 (from 2)
Receiving objects: 100% (460/460), 42.89 MiB | 11.71 MiB/s, done.
Resolving deltas: 100% (230/230), done.


## (Solo ejecutar si quieres guardar los resultados en TU google drive)  googledrive montado en local

In [ ]:
#conecar a google drive para guardar alli los outputs que quiera mantener (y que no se me olvide descargarlos) luego los subiré a Git para su uso posterior
from google.colab import drive
import os

# Montar Google Drive si aún no está montado
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Define el path de salida en Google Drive
output_path = '/content/drive/MyDrive/Reto21dias_24/'

# Verifica si el directorio existe y si no, lo crea
if not os.path.exists(output_path):
    try:
        os.makedirs(output_path)
        print(f"Directorio creado: {output_path}")
    except Exception as e:
        print(f"Error al crear el directorio: {e}")
else:
    print(f"El directorio ya existe: {output_path}")

MessageError: Error: credential propagation was unsuccessful

##Ruta a los archivos cargados y guardados en raiz de Colab (si se corta la conexion sin descargarlos se pierden)

In [1]:
# Define el nombre del archivo y el path de entrada y salida en raiz de google colab

# estructura del archivo de datos input_file: # Component	Description	Type	Class
# input_file_name = 'ACEDE-ECN-workshop-dic-2024-HIWP3.csv'
# input_file_name = 'ACEDE-ECN-workshop-dic-2024-HIWP.xlsx'
input_file_name = 'UBschoolMay25-dataCribadoTransformersNewCat-SHORT.xlsx'

input_path = '/content/'

output_path = '/content/'

Parametrización de variables


In [2]:
import os # Import the os module at the top of your script
# Construye el path completo
full_in_path = os.path.join(input_path, input_file_name)
full_in_path
# full_out_path = os.path.join(output_path, file_name)

# Define la variable con el nombre de la columna
target_column = 'Description'

# Objetos a clasificar
selec_objects = ['art1','art']
# selec_objects = ['cat1']

#categorias en la que quiero clasificar los objetos
# selec_categories = ['cat3']
selec_categories = ['cat1', 'cat2']

# el identificador [component] de la categoria que quiero mostrar en el resumen de clasificación global

# target_category ='promptB'
target_category ='GOAL'


# prueba (en desarrollo) para leer diferentes formatos de csv (no comprobada)

In [ ]:
import pandas as pd

def leer_archivo(ruta_archivo, **kwargs):
    extension = ruta_archivo.split('.')[-1].lower()

    try:
        if extension in ['xlsx', 'xls']:
            return pd.read_excel(ruta_archivo, **kwargs)
        elif extension == 'csv':
            # Lista de configuraciones a intentar
            configs = [
                {'encoding': 'utf-8', 'sep': ','},
                {'encoding': 'latin1', 'sep': ';'},
                {'encoding': 'utf-8', 'sep': ';'},
                {'encoding': 'cp1252', 'sep': ';'}  # Común en Windows
            ]

            # Intentar cada configuración
            for config in configs:
                try:
                    # Combinar configuración predeterminada con kwargs
                    params = {**config, **kwargs}
                    return pd.read_csv(ruta_archivo, **params)
                except:
                    continue

            raise ValueError("No se pudo leer el archivo CSV con ninguna configuración")
        else:
            raise ValueError(f"Formato de archivo no soportado: {extension}")

    except Exception as e:
        print(f"Error al leer el archivo: {str(e)}")
        return None

# Ejemplos de uso:
# Lectura básica
hiwp_data = leer_archivo(full_in_path)

# Lectura con parámetros específicos
# hiwp_data = leer_archivo(full_in_path, decimal=',', thousands='.')
# hiwp_data = leer_archivo(full_in_path, sheet_name='Hoja1')  # Para Excel

# leer el archivo

In [3]:
import pandas as pd
# Leer el archivo CSV o EXCEL
# Originalmente era un CSV de Open Office porque si lo generaba con MSexcel no se mapeaba bien )
    # Para guardar correctamente un archivo CSV con comas desde OpenOffice cuando viene de Excel con punto y coma (;), sigue estos pasos:
    # En OpenOffice, ve a "Archivo → Guardar como"
    # Selecciona "Texto CSV (.csv)" como tipo de archivo
    # Marca la casilla "Editar configuración de filtro"
    # En la ventana de configuración:
    # Juego de caracteres: Unicode (UTF-8)
    # Delimitador de campo: selecciona "," (coma)
    # Delimitador de texto: " (comillas)
    # NO se marca tener todas las cadenas de texto con ""
#mejoro el codigo con esta funcion que debe gestionar automático el formato de CSV o de excel (la primera de las hojas del libro)

def leer_archivo(ruta_archivo):
    # Obtener la extensión del archivo
    extension = ruta_archivo.split('.')[-1].lower()

    try:
        if extension in ['xlsx', 'xls']:
            # Leer archivo Excel
            return pd.read_excel(ruta_archivo)
        elif extension == 'csv':
            # Intentar primero con encoding UTF-8 y separador ','
            try:
                return pd.read_csv(ruta_archivo, encoding='utf-8')
            except:
                # Si falla, intentar con encoding latin1 y separador ';'
                try:
                    return pd.read_csv(ruta_archivo, encoding='latin1', sep=';')
                except:
                    # Último intento con encoding utf-8 y separador ';'
                    return pd.read_csv(ruta_archivo, encoding='utf-8', sep=';')
        else:
            raise ValueError(f"Formato de archivo no soportado: {extension}")

    except Exception as e:
        print(f"Error al leer el archivo: {str(e)}")
        return None

# Uso de la funcion
hiwp_data = leer_archivo(full_in_path)

# hiwp_data = leer_archivo('/content/ART-749-dataCribadoTransformers.xlsx')
# hiwp_data = pd.read_excel('/content/ART-749-dataCribadoTransformers.xlsx')
hiwp_data
# Verificar si la lectura fue exitosa
if hiwp_data is not None:
    print("Archivo leído correctamente")
    print(f"Dimensiones del DataFrame: {hiwp_data.shape}")
else:
    print("No se pudo leer el archivo")

# Mostrar el DataFrame resultante
# print(hiwp_data)

# Crear el DataFrame 'objects'
objects = hiwp_data[hiwp_data['Type'].isin(selec_objects)]

# Crear el DataFrame 'categories'
categories = hiwp_data[hiwp_data['Type'].isin(selec_categories)]

# Opción 1: Reconstruir el índice descartando el índice original
# objects= objects.reset_index(drop=True)
# categories= categories.reset_index(drop=True)

# Opción 2: Reconstruir el índice manteniendo el índice original como una nueva columna
objects= objects.reset_index()
categories= categories.reset_index()


# Verificar los resultados
print("DataFrame 'objects':")
print(objects.shape)
print(objects['Type'].value_counts())
print("\nPrimeras filas de 'objects':")
print(objects.head())

print("\n" + "="*50 + "\n")

print("DataFrame 'categories':")
print(categories.shape)
print(categories['Type'].value_counts())
print("\nPrimeras filas de 'categories':")
print(categories.head())

Archivo leído correctamente
Dimensiones del DataFrame: (202, 3)
DataFrame 'objects':
(194, 4)
Type
art    194
Name: count, dtype: int64

Primeras filas de 'objects':
   index Component                                        Description Type
0      8  id0001#0   Study on Data Analysis of Assessment in Class...  art
1      9  id0012#0   Flipped pedagogy and student evaluations of t...  art
2     10  id0021#0   ntegrity of Attitudes and Rating Tendencies o...  art
3     11  id0022#2   Making Meaning from Student Evaluations of Te...  art
4     12  id0024#0   Student evaluations of teaching (SET): Guidel...  art


DataFrame 'categories':
(8, 4)
Type
cat1    4
cat2    4
Name: count, dtype: int64

Primeras filas de 'categories':
   index Component                                        Description  Type
0      0      GOAL  Student evaluation of teaching open-ended comm...  cat1
1      1     GOAL1  Studies that focus on examining the free-text ...  cat1
2      2   GOAL1sp  estudios que se enf


# CODIGO para Embeedings




In [4]:
from importlib import metadata
import subprocess
import sys
import importlib

required_packages = [
    'seaborn',
    'matplotlib',
    'sentence-transformers',
    'pandas',
    'numpy',
    'transformers',
    'torch',
    'xlrd',
    'openpyxl'
]

def get_installed_packages():
    """Obtiene una lista de paquetes instalados usando importlib.metadata"""
    try:
        return {dist.metadata['Name'].lower() for dist in metadata.distributions()}
    except Exception:
        return set()

def install_packages():
    installed_packages = get_installed_packages()
    packages_to_install = [
        pkg for pkg in required_packages
        if pkg.lower().replace('-', '_') not in installed_packages
    ]

    if packages_to_install:
        print("Instalando paquetes faltantes...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages_to_install)
            print("\nTodos los paquetes faltantes han sido instalados.")
        except subprocess.CalledProcessError as e:
            print(f"\nError durante la instalación: {e}")
            return False
    else:
        print("Todos los paquetes requeridos ya están instalados.")

    return True

def verify_installations():
    print("\nVerificando instalaciones:")
    all_installed = True
    for package in required_packages:
        try:
            # Intentar importar el módulo
            importlib.import_module(package.replace('-', '_'))
            # Obtener la versión instalada
            version = metadata.version(package)
            print(f"✓ {package} (versión {version})")
        except (ImportError, metadata.PackageNotFoundError):
            print(f"✗ {package} - No se pudo importar")
            all_installed = False
    return all_installed

def load_required_imports():
    """Función separada para cargar las importaciones necesarias"""
    try:
        # Realizar las importaciones globalmente
        global SentenceTransformer, util, pd, np, torch, time
        from sentence_transformers import SentenceTransformer, util
        import pandas as pd
        import numpy as np
        import torch
        import time
        print("\nTodas las importaciones completadas con éxito.")
        return True
    except ImportError as e:
        print(f"\nError al importar las bibliotecas: {e}")
        return False

def main():
    # Ejecutar instalación y verificación
    if install_packages() and verify_installations():
        print("\nProcediendo con las importaciones...")
        return load_required_imports()
    else:
        print("\nHubo problemas con la instalación o verificación de paquetes.")
        return False

if __name__ == "__main__":
    success = main()
    if success:
        # Aquí ya puedes usar SentenceTransformer y demás imports
        print("Sistema listo para usar")

Instalando paquetes faltantes...

Todos los paquetes faltantes han sido instalados.

Verificando instalaciones:
✓ seaborn (versión 0.13.2)
✓ matplotlib (versión 3.10.0)
✓ sentence-transformers (versión 4.1.0)
✓ pandas (versión 2.2.2)
✓ numpy (versión 2.0.2)
✓ transformers (versión 4.53.0)
✓ torch (versión 2.6.0+cu124)
✓ xlrd (versión 2.0.2)
✓ openpyxl (versión 3.1.5)

Procediendo con las importaciones...

Todas las importaciones completadas con éxito.
Sistema listo para usar


In [5]:
from datetime import datetime
# Obtener la fecha y hora actual para etiquear hora del analisis en los nombres de archivo
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
# Lista de modelos a probar
# Seleccion de los 1 de los rapidos entre los 5 mejores del articulo WPOM 2024
# models_to_test = [
#     'all-MiniLM-L6-v2'
# ]

# todos los modelos
# models_to_test = [
#     'all-MiniLM-L6-v2',
#     'all-distilroberta-v1',
#     'all-mpnet-base-v2',
#     'paraphrase-multilingual-mpnet-base-v2',
#     'distiluse-base-multilingual-cased-v1',
#     'all-MiniLM-L12-v2',
#     'allenai-specter',
#     'allenai/scibert_scivocab_uncased',
#     'distilbert-base-nli-mean-tokens',
#     'roberta-base-nli-stsb-mean-tokens',
#     'distiluse-base-multilingual-cased-v2',
#     'paraphrase-multilingual-MiniLM-L12-v2',
#     'stsb-roberta-large',
#     'bert-base-nli-mean-tokens'
#     'BAAI/bge-large-en-v1.5',    # 1024D - Top choice
#     'intfloat/e5-large-v2',      # 1024D - Alternativa sólida
#     'sentence-transformers/gtr-t5-large',      # Bueno para académicos
#     'allenai/specter2_proximity' #ideal para cribado de revisiones sistemáticas de literatura (SLR), ya que está específicamente diseñado para trabajar con documentos científicos.
# ]

# Seleccion de los 5 mejores del articulo WPOM 2024
models_to_test = [
    'all-MiniLM-L6-v2'
    # 'all-distilroberta-v1',
    # 'all-mpnet-base-v2',
    # 'all-MiniLM-L12-v2',
    # 'allenai-specter',
    # 'BAAI/bge-large-en-v1.5',    # 1024D - Top choice
    #  'allenai/specter2_proximity' # pendiente de parametrizar para que funcione specter ideal para cribado de revisiones sistemáticas de literatura (SLR), ya que está específicamente diseñado para trabajar con documentos científicos.

]

# Seleccion de los peores del articulo WPOM 2024
# models_to_test = [
#     'paraphrase-multilingual-mpnet-base-v2',
#     'distiluse-base-multilingual-cased-v1',
#     'allenai/scibert_scivocab_uncased',
#     'distilbert-base-nli-mean-tokens',
#     'roberta-base-nli-stsb-mean-tokens',
#     'distiluse-base-multilingual-cased-v2',
#     'paraphrase-multilingual-MiniLM-L12-v2',
#     'stsb-roberta-large',
#     'bert-base-nli-mean-tokens'
# ]

# Función para evaluar un modelo
def evaluate_model(model_name, objects_df, categories_df):
    print(f"Evaluating model: {model_name}")
    model = SentenceTransformer(model_name)

    start_time = time.time()

    # Generar embeddings para objetos
    object_texts = objects_df[target_column].tolist()
    object_embeddings = model.encode(object_texts, show_progress_bar=True)

    # Generar embeddings para categorías
    category_texts = categories_df[target_column].tolist()
    category_embeddings = model.encode(category_texts, show_progress_bar=True)

    # Calcular similitud coseno
    similarity_matrix = util.cos_sim(object_embeddings, category_embeddings)

    # Encontrar las mejores coincidencias
    best_matches = np.argmax(similarity_matrix, axis=1)

    end_time = time.time()
    processing_time = end_time - start_time

    return {
        'model': model_name,
        'best_matches': categories_df.iloc[best_matches]['Component'].tolist(),
        'processing_time': round(processing_time, 2),  # Redondear a 2 decimales
        'embedding_size': object_embeddings.shape[1],
        'similarity_matrix': similarity_matrix,
        'object_embeddings': object_embeddings,
        'category_embeddings': category_embeddings
    }

# Evaluar todos los modelos
results = []
similarity_data = []
embeddings_data = []

for model_name in models_to_test:
    try:
        result = evaluate_model(model_name, objects, categories)
        results.append({
            'model': result['model'],
            'best_matches': result['best_matches'],
            'processing_time': result['processing_time'],
            'embedding_size': result['embedding_size']
        })

        # Preparar datos de similitud para este modelo
        for i, obj in enumerate(objects['Component']):
            row = [model_name, obj] + result['similarity_matrix'][i].tolist()
            similarity_data.append(row)

        # Preparar datos de embeddings para este modelo
        for i, obj in enumerate(objects['Description']):
            embeddings_data.append([model_name, objects['Component'].iloc[i], obj, result['object_embeddings'][i].tolist()])
        for i, cat in enumerate(categories['Description']):
            embeddings_data.append([model_name, categories['Component'].iloc[i], cat, result['category_embeddings'][i].tolist()])

    except Exception as e:
        print(f"Error evaluating {model_name}: {str(e)}")

# Crear DataFrame con resultados
df_results = pd.DataFrame(results)
df_results = df_results.rename(columns={'processing_time': 'processing_time_seconds'})
print(df_results)

# Crear DataFrame de similitud coseno
columns = ['Model', 'Object'] + categories['Component'].tolist()
df_similarity = pd.DataFrame(similarity_data, columns=columns)

# Crear DataFrame de embeddings
df_embeddings = pd.DataFrame(embeddings_data, columns=['Model', 'Component', 'Description', 'Embedding'])

# Crear la nueva tabla de similitud para HIWPshortDescription
df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len(models_to_test)
df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])


Evaluating model: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

              model                                       best_matches  \
0  all-MiniLM-L6-v2  [SET, SET, SET, SET, SET, SET, SET, SET, SET, ...   

   processing_time_seconds  embedding_size  
0                   108.58             384  


### opcional para modelos de alta dimensionalidad

In [6]:
# Instalar PEFT para SPECTER2
!pip install peft accelerate

# Importar librerías adicionales
from transformers import AutoTokenizer, AutoModel
import torch

# Lista de modelos a probar - NUEVOS MODELOS DE ALTA DIMENSIONALIDAD
# Comentarios sobre cada modelo para tu contexto de Management/Business:

models_to_test = [
    # 'BAAI/bge-large-en-v1.5',    # 1024D - Top choice para embeddings densos
    # 'intfloat/e5-large-v2',      # 1024D - Alternativa sólida, excelente similitud semántica
    # 'sentence-transformers/gtr-t5-large'      # Bueno para académicos, arquitectura T5
    'allenai-specter',                    # SPECTER1 - más estable para SLR
    'allenai/specter2_proximity' # Ideal para cribado SLR, específico para documentos científicos
]

# Si quieres comparar con tu baseline actual, descomenta esta línea:
models_to_test.append('all-MiniLM-L6-v2')  # Tu modelo actual para comparación

# Función para evaluar un modelo (optimizada para modelos grandes)
def evaluate_model(model_name, objects_df, categories_df):
    print(f"Evaluating model: {model_name}")

    # Configuración específica para modelos grandes en Colab
    torch.cuda.empty_cache()  # Limpiar memoria GPU si está disponible

    try:
        # Cargar modelo con configuraciones optimizadas para Colab
        if 'specter2' in model_name:
            print("Cargando SPECTER2 con configuración especial...")
            try:
                # Método 1: Intentar carga directa con PEFT
                model = SentenceTransformer(model_name, trust_remote_code=True)
                print("SPECTER2 cargado correctamente con PEFT")
            except Exception as e1:
                print(f"Error método 1: {e1}")
                try:
                    # Método 2: Cargar componentes por separado
                    print("Intentando carga alternativa...")
                    from transformers import AutoTokenizer, AutoModel
                    tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base', trust_remote_code=True)
                    base_model = AutoModel.from_pretrained('allenai/specter2_base', trust_remote_code=True)

                    # Crear SentenceTransformer a partir del modelo base
                    from sentence_transformers import models
                    word_embedding_model = models.Transformer(model_name_or_path='allenai/specter2_base')
                    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
                    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
                    print("SPECTER2 cargado con método alternativo")
                except Exception as e2:
                    print(f"Error método 2: {e2}")
                    # Método 3: Fallback a SPECTER1
                    print("Fallback a allenai-specter (versión anterior)")
                    model = SentenceTransformer('allenai-specter')
                    model_name = 'allenai-specter'  # Actualizar nombre para resultados
        else:
            # Cargar otros modelos normalmente
            model = SentenceTransformer(model_name)

        # Usar device automático (GPU si está disponible, sino CPU)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model = model.to(device)

        print(f"Modelo cargado en: {device}")
        print(f"Dimensiones del embedding: {model.get_sentence_embedding_dimension()}")

    except Exception as e:
        print(f"Error cargando modelo {model_name}: {str(e)}")
        return None

    start_time = time.time()

    try:
        # Generar embeddings para objetos con batch size optimizado para Colab
        object_texts = objects_df[target_column].tolist()
        object_embeddings = model.encode(
            object_texts,
            show_progress_bar=True,
            batch_size=4,  # Reducido para modelos grandes
            convert_to_tensor=True,
            device=device
        )

        # Generar embeddings para categorías
        category_texts = categories_df[target_column].tolist()
        category_embeddings = model.encode(
            category_texts,
            show_progress_bar=True,
            batch_size=4,  # Reducido para modelos grandes
            convert_to_tensor=True,
            device=device
        )

        # Calcular similitud coseno
        similarity_matrix = util.cos_sim(object_embeddings, category_embeddings)

        # Convertir a numpy para compatibilidad con el resto del código
        if torch.is_tensor(similarity_matrix):
            similarity_matrix = similarity_matrix.cpu().numpy()
        if torch.is_tensor(object_embeddings):
            object_embeddings = object_embeddings.cpu().numpy()
        if torch.is_tensor(category_embeddings):
            category_embeddings = category_embeddings.cpu().numpy()

        # Encontrar las mejores coincidencias
        best_matches = np.argmax(similarity_matrix, axis=1)

        end_time = time.time()
        processing_time = end_time - start_time

        # Limpiar memoria después de cada modelo
        del model
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

        return {
            'model': model_name,
            'best_matches': categories_df.iloc[best_matches]['Component'].tolist(),
            'processing_time': round(processing_time, 2),
            'embedding_size': object_embeddings.shape[1],
            'similarity_matrix': similarity_matrix,
            'object_embeddings': object_embeddings,
            'category_embeddings': category_embeddings
        }

    except Exception as e:
        print(f"Error durante el procesamiento con {model_name}: {str(e)}")
        # Limpiar memoria en caso de error
        try:
            del model
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
        except:
            pass
        return None

# Evaluar todos los modelos (resto del código igual)
results = []
similarity_data = []
embeddings_data = []

for model_name in models_to_test:
    print(f"\n{'='*60}")
    print(f"Procesando modelo: {model_name}")
    print(f"{'='*60}")

    try:
        result = evaluate_model(model_name, objects, categories)

        if result is not None:  # Solo procesar si el modelo se evaluó correctamente
            results.append({
                'model': result['model'],
                'best_matches': result['best_matches'],
                'processing_time': result['processing_time'],
                'embedding_size': result['embedding_size']
            })

            # Preparar datos de similitud para este modelo
            for i, obj in enumerate(objects['Component']):
                row = [model_name, obj] + result['similarity_matrix'][i].tolist()
                similarity_data.append(row)

            # Preparar datos de embeddings para este modelo
            for i, obj in enumerate(objects['Description']):
                embeddings_data.append([model_name, objects['Component'].iloc[i], obj, result['object_embeddings'][i].tolist()])
            for i, cat in enumerate(categories['Description']):
                embeddings_data.append([model_name, categories['Component'].iloc[i], cat, result['category_embeddings'][i].tolist()])

            print(f"✅ {model_name} completado exitosamente")
            print(f"   Dimensiones: {result['embedding_size']}D")
            print(f"   Tiempo: {result['processing_time']}s")
        else:
            print(f"❌ {model_name} falló durante la evaluación")

    except Exception as e:
        print(f"❌ Error crítico evaluando {model_name}: {str(e)}")

# Crear DataFrame con resultados (resto del código igual)
if results:  # Solo crear DataFrame si hay resultados
    df_results = pd.DataFrame(results)
    df_results = df_results.rename(columns={'processing_time': 'processing_time_seconds'})
    print(f"\n{'='*60}")
    print("RESUMEN DE RESULTADOS:")
    print(f"{'='*60}")
    print(df_results)

    # Crear DataFrame de similitud coseno
    columns = ['Model', 'Object'] + categories['Component'].tolist()
    df_similarity = pd.DataFrame(similarity_data, columns=columns)

    # Crear DataFrame de embeddings
    df_embeddings = pd.DataFrame(embeddings_data, columns=['Model', 'Component', 'Description', 'Embedding'])

    # Crear la nueva tabla de similitud para target_category
    df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
    df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
    df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len([r['model'] for r in results])
    df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
    df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])

    print(f"\n✅ Procesamiento completado para {len(results)} modelos")
else:
    print("❌ No se pudieron evaluar ninguno de los modelos")


Procesando modelo: allenai-specter
Evaluating model: allenai-specter


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo cargado en: cpu
Dimensiones del embedding: 384


Batches:   0%|          | 0/49 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ all-MiniLM-L6-v2 completado exitosamente
   Dimensiones: 384D
   Tiempo: 24.08s

RESUMEN DE RESULTADOS:
                        model  \
0             allenai-specter   
1  allenai/specter2_proximity   
2            all-MiniLM-L6-v2   

                                        best_matches  processing_time_seconds  \
0  [SET, SET, GOAL2, GOAL, SET, GOAL1, SET, SET, ...                   179.76   
1  [SET, SET, GOAL1, GOAL1, SET, GOAL1, GOAL2, SE...                   173.82   
2  [SET, SET, SET, SET, SET, SET, SET, SET, SET, ...                    24.08   

   embedding_size  
0             768  
1             768  
2             384  

✅ Procesamiento completado para 3 modelos


## continuamos con el codigo normal

In [ ]:
###OPCIONAL##### SOLO EJECUTAR para reahacer el analisis de proximidad con otra categoria como target sin tener que repetir los embeedings
# el identificador [component] de la categoria que quiero mostrar en el resumen de clasificación global
# target_category ='HIWPshortDescrip'
# target_category ='Empowerment'
# target_category ='WorkLifeBalance'
# target_category ='RemoteWork'
# target_category ='GrenHRM'
# target_category ='OpMange'
# target_category ='KaizenLong'
# target_category ='KaizenCulture'
# target_category ='promptA'
# target_category ='SLR'
target_category ='OpenEnded'
# promptAscag promptAperf promptBscag promptBperfCa promptBperf promptBca

# Crear la nueva tabla de similitud para la target_category
df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len(models_to_test)
df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])




In [7]:
# Calcula el rango de similitud para cada modelo individualmente, donde el rango 1 es el mejor (mayor similitud).
# Calcula el rango promedio para cada Object_Description a través de todos los modelos.
# Ordena los objetos por su rango promedio (el más bajo es el mejor).
# La tabla resumen final contendrá las siguientes columnas:
# # Object_Component: El componente del objeto.
# # Object_Description: La descripción del objeto.
# # Average_Rank: El rango promedio del objeto a través de todos los modelos (más bajo es mejor).
# Los objetos estarán ordenados por su rango promedio,
# lo que  dará una visión integrada de cómo se comportan los objetos a clasificar, a través de todos los modelos,
# en relación con la categoria sobre la que me interesa clasificarlos.

# No es sencillo poner un punto de corte a partir del cual se consideran "excluidos" los objetos a clasificar.
# de modo que se haría un escreening por titulo y abstract manual.
# partiendo del orden de esta tabla hasta llegar aun  punto donde # haya varios seguidos sin seleccionar.
# Momento en el que se podría parar el screening asistido por clasificacion automática.

# Función para calcular el rango inverso (el más alto obtiene el rango 1)
def inverse_rank(series):
    return series.rank(ascending=False, method='min')

# Calcular rangos para cada modelo
df_ranks = df_hiwp_similarity.groupby('Model').apply(lambda x: x.assign(Rank=inverse_rank(x['Similarity']))).reset_index(drop=True)

# Calcular el rango promedio para cada Object_Component a través de todos los modelos
df_summary = df_ranks.groupby('Object_Component')['Rank'].mean().reset_index()
df_summary = df_summary.rename(columns={'Rank': 'Average_Rank'})

# Añadir la Description correspondiente a cada Component
df_summary = df_summary.merge(objects[['Component', 'Description']], left_on='Object_Component', right_on='Component', how='left')

# Ordenar por rango promedio (ascendente, ya que el rango más bajo es mejor)
df_summary = df_summary.sort_values('Average_Rank')

# Reordenar las columnas
df_summary = df_summary[['Object_Component', 'Description', 'Average_Rank']]

# Mostrar las primeras filas de la tabla resumen
print(target_category)
print(df_summary.head(20))

GOAL
    Object_Component                                        Description  \
53          id0343#1   Student evaluation of teaching: reactions of ...   
52          id0340#1   Stress, distress, disorder and coping: the im...   
132         id0526#1   Measuring Student Responses in and Instructor...   
6           id0069#0   Student Evaluations of Teaching Tools A Quali...   
46          id0319#1   Appearance, insults, allegations, blame and t...   
144         id0562#1   Teachers' experiences of student feedback: A ...   
108         id0471#1   Beyond satisfaction scores: visualising stude...   
86          id0403#1   Themes from Veterinary Student Evaluations of...   
162         id0604#1   The potential in student evaluation of teachi...   
186         id0665#0   Teaching Through the Student Lens: Qualitativ...   
38          id0290#1   When students' words hurt: 12 tips for helpin...   
189         id0692#1   How Does Students' Negative Feedback Affect U...   
155         id0586#1

/tmp/ipython-input-7-4037176016.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranks = df_hiwp_similarity.groupby('Model').apply(lambda x: x.assign(Rank=inverse_rank(x['Similarity']))).reset_index(drop=True)


In [8]:
# etiquetar los archivos con alguna información adcional
# tag_file = tag_file
# tag_file = 'best5 models'
# tag_file = 'worst9 models'
# tag_file = ''
# tag_file = 'all models-articles'
tag_file = 'DOEJUL25-DosModelos_spectet2_all-MiniLM-L6-v2'

In [9]:
# código para guardar todos los DataFrames en un único archivo Excel, con cada DataFrame en una hoja separada

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Asegurarse de que la carpeta de salida existe
os.makedirs(output_path, exist_ok=True)

# Crear el nombre del archivo Excel
excel_filename = f'classification_{tag_file}_{target_category}_{current_time}.xlsx'
excel_path = os.path.join(output_path, excel_filename)

# Crear un ExcelWriter object
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    # Guardar cada DataFrame en una hoja separada
    df_results.to_excel(writer, sheet_name='model_comparison_results', index=False)
    df_similarity.to_excel(writer, sheet_name='tablas_similitud_coseno', index=False)
    df_embeddings.to_excel(writer, sheet_name='embeddings', index=False)
    df_hiwp_similarity.to_excel(writer, sheet_name='hiwp_similarity', index=False)
    df_summary.to_excel(writer, sheet_name='hiwp_similarity_summary', index=False)

print(f"Todos los resultados han sido guardados en {excel_path}")

Todos los resultados han sido guardados en /content/classification_DOEJUl25-DosModelos_spectet2_all-MiniLM-L6-v2_GOAL_20250707_092045.xlsx


# OPCIONAL. un código que lea los embeddings del archivo Excel y los prepare para poder reutilizarlos con los chunks de codigo de...

ACEDE-ECN-dic24-cribado_LLMhiwp_embeddingsv2-shared.ipynb
(basado en ACEDE_ECN_dic24_cribado_LLMhiwp_embeddingsv2_shared.ipynb)

Sin tnere que volver a calcular los embeddigns, que según el numero de objeto y modelo puedes consumir mucho tiempo (por ejemplo con 1024 tolens y 1200 objetos a vectorizar son unos 30 minutos, con un modelo de 384 tokesn son unos 3 minutos solamente)

El dato de entrada datosembeddings.xlsx es el resultdo que guarda cualquiera de esos dos códigos

TEngo que terener cargados los dataframe de objects y categories...

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import util
import ast

# Leer el archivo Excel
# Para archivo .xls
# df_embeddings = pd.read_excel('datosembeddings.xls', sheet_name='embeddings', engine='xlrd')

# Para archivo .xlsx
df_embeddings = pd.read_excel('ART-749-embeddings.xlsx', sheet_name='embeddings', engine='openpyxl')

# Obtener la lista de modelos únicos del DataFrame de embeddings
models_to_test = df_embeddings['Model'].unique().tolist()

# Convertir los embeddings de string a lista de números
df_embeddings['Embedding'] = df_embeddings['Embedding'].apply(ast.literal_eval)

# Crear diccionario para almacenar los resultados
results = []
similarity_data = []

# Procesar cada modelo
for model_name in df_embeddings['Model'].unique():
    # Filtrar datos para el modelo actual
    model_data = df_embeddings[df_embeddings['Model'] == model_name]

    # Separar embeddings de objetos y categorías
    objects_data = model_data[model_data['Component'].isin(objects['Component'])]
    categories_data = model_data[model_data['Component'].isin(categories['Component'])]

    # Convertir embeddings a arrays numpy
    object_embeddings = np.array(objects_data['Embedding'].tolist())
    category_embeddings = np.array(categories_data['Embedding'].tolist())

    # Calcular similitud coseno
    similarity_matrix = util.cos_sim(object_embeddings, category_embeddings)

    # Encontrar las mejores coincidencias
    best_matches = np.argmax(similarity_matrix, axis=1)

    # Guardar resultados
    results.append({
        'model': model_name,
        'best_matches': categories.iloc[best_matches]['Component'].tolist(),
        'embedding_size': object_embeddings.shape[1]
    })

    # Preparar datos de similitud para este modelo
    for i, obj in enumerate(objects['Component']):
        row = [model_name, obj] + similarity_matrix[i].tolist()
        similarity_data.append(row)

# Crear DataFrame con resultados
df_results = pd.DataFrame(results)

# Crear DataFrame de similitud coseno
columns = ['Model', 'Object'] + categories['Component'].tolist()
df_similarity = pd.DataFrame(similarity_data, columns=columns)

# A partir de aquí puedes usar el código que ya tenías para crear df_hiwp_similarity
df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len(models_to_test)
df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])

# continua con ###OPCIONAL##### SOLO EJECUTAR para reahacer el analisis de proximidad con otra categoria como target sin tener que repetir los embeedings
# o con # Calcula el rango de similitud para cada modelo individualmente, donde el rango 1 es el mejor (mayor similitud).
